## Demo

In [1]:
from models import OOIDataset

In [2]:
# This will load the dataset by reading through the zarr meta and load as dask arrays
ooi_ds = OOIDataset(dataset_id="RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample")

In [3]:
ooi_ds

<RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample: 543.6 GB>
Dimensions: (time)
Data variables: 
    bottom_pressure
    bottom_pressure_qc_executed
    bottom_pressure_qc_results
    deployment
    driver_timestamp
    ingestion_timestamp
    internal_timestamp
    port_timestamp
    press_trans_temp
    time

In [4]:
start_idx = ooi_ds.time.data.chunksize[0] * 401 * 2

# Grab the dimension array
# As of 6/14/2023, @lsetiawan and @dcherian found that there are some issues
# in the dataset, so we'll just grab half the data,
# seems like there are some repetitions that happened after chunk 401
time = ooi_ds.time[start_idx:]

time

<xarray.DataArray 'time' (time: 440101366)>
dask.array<getitem, shape=(440101366,), dtype=float64, chunksize=(12000000,), chunktype=numpy.ndarray>
Dimensions without coordinates: time
Attributes:
    axis:           T
    calendar:       gregorian
    long_name:      time
    standard_name:  time
    units:          seconds since 1900-01-01 0:0:0

In [5]:
data_vars = {}
for k, v in ooi_ds.variables.items():
    if k not in ooi_ds.dimensions:
        data_vars[k] = v[start_idx:]

## Create dataset 

Now try to create dataset here and the index

In [6]:
from typing import Any, Hashable, Mapping

import dask
import numpy as np
import pandas as pd
import xarray as xr
from dask.utils import memory_repr
from xarray.core.indexes import Index, PandasIndex
from xarray.core.variable import Variable

In [7]:
import dask
import numpy as np

In [8]:
from dask.distributed import Client

In [9]:
client = Client()

In [10]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:65534,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:49161,Total threads: 2
Dashboard: http://127.0.0.1:49167/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:49153,


In [19]:
class DaskIndex(Index):
    def __init__(self, array: Any, dim: Hashable, coord_dtype: Any = None, attrs=None):
        self.attrs = attrs
        # Create lookup table
        index = self._create_lookup(array)

        if index.name is None:
            index.name = dim

        self.index = index
        self.dim = dim

        self.coord_dtype = coord_dtype

    def _decode_times(self, values):
        if self.attrs and "calendar" in self.attrs:
            units = self.attrs.get("units")
            calendar = self.attrs.get("calendar")

            return xr.coding.times.decode_cf_datetime(values, units, calendar)

        return values

    def _create_lookup(self, array):
        # Code by @dcherian
        chunks = array.chunks
        summed = np.cumsum(chunks)
        ends = summed - 1
        starts = np.insert(summed[:-1], 0, 0)

        interleaved = np.vstack([starts, ends]).T.reshape(-1)

        (computed,) = dask.compute(array[interleaved])
        
        # TODO: this decoding should be unnecessary once we let
        # xarray open datasets without creating indexes
        known_starts = self._decode_times(computed[::2])
        known_ends = self._decode_times(computed[1::2])

        # TODO: can we do .from_breaks(known_starts)?
        self._lookup = pd.IntervalIndex.from_tuples(
            tuple(zip(known_starts, known_ends)), closed="both"
        )

        return array

    def _replace(self, index, dim=None, coord_dtype=None):
        if dim is None:
            dim = self.dim
        if coord_dtype is None:
            coord_dtype = self.coord_dtype
        return type(self)(index, dim, coord_dtype)

    # TODO: see notes about IndexSelResult
    #    The latter is a small class that stores positional indexers (indices)
    #    and that could also store new variables, new indexes,
    #    names of variables or indexes to drop,
    #    names of dimensions to rename, etc.
    def sel(self, indexers, **kwargs):
        dim = self.dim
        # sel needs to only handle keys in labels
        # since it delegates to isel.
        # we handle all entries in ._indexes there
        results = []
        for k in indexers:
            if k == dim:
                indxr = indexers[dim]
                if isinstance(indxr, str):
                    indxr = [indxr]
                if isinstance(indxr, slice):
                    indxr = [indxr.start, indxr.stop]

                # load all chunks that enclose the range
                chunkidx = np.unique(self._lookup.get_indexer(indxr))
                # create a new pd.Index
                new_pandas_index = pd.Index(
                    self._decode_times(self.index.blocks[chunkidx].compute())
                )
                
                # BAD DATA!
                #assert new_pandas_index.is_monotonic_increasing

                # Just use existing implementation to figure out what to return
                results = PandasIndex(new_pandas_index, dim=dim).sel({dim: indexers[dim]})
                print(results)
                return results

    def isel(self, indexers):
        print("calling isel")
        print(indexers)
        indxr = indexers[self.dim]

        # TODO: second load, wasteful!
        new_pandas_index = pd.Index(self._decode_times(self.index[indxr].compute()))
        # TODO: Change this so it returns same index??? Not sure how to do this
        return PandasIndex(new_pandas_index, dim=self.dim)

    @classmethod
    def from_variables(
        cls,
        variables: Mapping[Any, Variable],
        *,
        options: Mapping[str, Any],
    ) -> "DaskIndex":
        name, var = next(iter(variables.items()))
        dim = var.dims[0]
        attrs = var.attrs

        data = var._data
        attrs = var._attrs
        obj = cls(data, dim, coord_dtype=var.dtype, attrs=attrs)
        # obj.index.name = name

        return obj

In [20]:
from xarray.tests import raise_if_dask_computes

# Create dataset, change the `time` array name to `dim_var1` and rename back
# this way, xarray doesn't automatically load the index as Pandas
# @lsetiawan Thoughts: maybe there's a flag when creating dataset like this to not
# auto create index with same name!
with raise_if_dask_computes():
    ds = xr.Dataset(
        data_vars=data_vars,
        coords={"dim_var1": time},
    ).rename({"dim_var1": "time"})
print(f"Total data size: {memory_repr(ds.nbytes)}")
ds

Total data size: 23.8 GB


/var/folders/yp/rzgd0f7n5zbcbf5dg73sfb8ntv3xxm/T/ipykernel_93850/2225275999.py:11: UserWarning: rename 'dim_var1' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ).rename({"dim_var1": "time"})


<xarray.Dataset>
Dimensions:                      (time: 440101366)
Coordinates:
    time                         (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
Data variables:
    bottom_pressure              (time) float32 dask.array<chunksize=(1000000,), meta=np.ndarray>
    bottom_pressure_qc_executed  (time) uint8 dask.array<chunksize=(76000000,), meta=np.ndarray>
    bottom_pressure_qc_results   (time) uint8 dask.array<chunksize=(76000000,), meta=np.ndarray>
    deployment                   (time) int32 dask.array<chunksize=(1000000,), meta=np.ndarray>
    driver_timestamp             (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    ingestion_timestamp          (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    internal_timestamp           (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    port_timestamp               (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    press_trans_temp             (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>

In [21]:
from xarray.tests import raise_if_dask_computes

# @dcherian : vendor this
with raise_if_dask_computes(max_computes=1):
    # one compute to create lookup table
    new_ds = ds.set_xindex("time", DaskIndex)
new_ds

<xarray.Dataset>
Dimensions:                      (time: 440101366)
Coordinates:
  * time                         (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
Data variables:
    bottom_pressure              (time) float32 dask.array<chunksize=(1000000,), meta=np.ndarray>
    bottom_pressure_qc_executed  (time) uint8 dask.array<chunksize=(76000000,), meta=np.ndarray>
    bottom_pressure_qc_results   (time) uint8 dask.array<chunksize=(76000000,), meta=np.ndarray>
    deployment                   (time) int32 dask.array<chunksize=(1000000,), meta=np.ndarray>
    driver_timestamp             (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    ingestion_timestamp          (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    internal_timestamp           (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    port_timestamp               (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
    press_trans_temp             (time) float64 dask.array<chunksize=(12000000,), meta=np.ndarray>
Indexes:
    time                         DaskIndex

In [ ]:
new_ds.sel(time="2023-06-01")

IndexSelResult(dim_indexers={'time': array([1981835, 1981836, 1981837, ..., 3709779, 3709780, 3709781])}, indexes={}, variables={}, drop_coords=[], drop_indexes=[], rename_dims={})
calling isel
{'time': array([1981835, 1981836, 1981837, ..., 3709779, 3709780, 3709781])}


In [24]:
new_ds.sel(time=slice("2023-06-01", "2023-06-10"))

KeyError: "cannot represent labeled-based slice indexer for coordinate 'time' with a slice over integer positions; the index is unsorted or non-unique"